In [1]:
import dlib
import cv2
import argparse
import os
from skimage import io
import numpy

In [2]:
class DlibFaceProcessor(object):
    
    def __init__(self, datapath, binarypath,mmod_model_path, shape_predictor_path, resnet_model_path):
        self.datapath = datapath
        self.binarypath = binarypath

        self.mmod_model_path = os.path.join(binarypath, mmod_model_path)
        self.shape_predictor_path = os.path.join(binarypath, shape_predictor_path)
        self.resnet_model_path = os.path.join(binarypath, resnet_model_path)
        
        #face detection, shape detection, face recognition
        slef.detector = dlib.cnn_face_detection_model_v1(mmod_model_path)
        self.shape_predictor = dlib.shape_predictor(self.shape_predictor_path)
        self.facerec = dlib.face_recognition_model_v1(self.resnet_model_path)
        
        #display image
        self.win = dlib.image_window()
        
        #parameter reset for each image
        self.reset()
    
    #reset list of descriptor and it's map for each image
    def reset(self):
        slef.list_of_images = []
        self.map_image_distance = {}
     
    #datapath to images
    def path_to_images(self):
        
        for root, dirs, files in os.walk(self.datapath):
            if(len(files) != 0):
                for file in files:   
                    file_path = root + os.sep + files
                    if file_path.endswith(".jpg"):
                        sefl.list_of_images.append(file_path)
                        
                        
    #reset 2: second method for reset-
#     list_of_images = [root + os.sep + file for root, dirs, files in os.walk(self.datapath) if (len(files) != 0) 
#         for file in files ]
    
    
    """
    process each image based on profile or Frontal pose for each person.
    1- read imag
    2- image key for 'map_imgage_distance'= file + profile/frontal
    """
    def process_images(self):
        
        for image in self.list_of_images:
            
            #read image from list
            img = io.imread(image)
            
            #assign a key for each image based on each face's name followed by profile or Frontal
            seperated_file = image.split("/")
            key_image = "_".join(seperated_file[-3:-1])
            #print(key_image, seperated_file)
            
            #create a key if it does not exist
            if key_image not in self.map_imgage_distance:
                map_image_distance[key_image] = []
             
            self.win.clear_overlay()
            self.win.set_image(img)
            
            
            dets = cnn_face_detector(img,1)
    
            rects = dlib.rectangles()
            rects.extend([d.rect for d in dets])
            print(rects)
            
            if len(dets) == 0: 
                print("face detector found {} in {} image. \n".format(len(dets),"_".join(seperated_path[-3:]) ))
        
        
    

In [3]:

class DlibFaceProcessor(object):

   def __init__(self, datapath, binarypath, predictor, resnet_model, display=False):
      self.datapath = datapath
      self.binarypath = binarypath

      self.predictor = os.path.join(binarypath, predictor)
      self.resnet_model = os.path.join(binarypath, resnet_model)

      self.detector = dlib.get_frontal_face_detector()
      self.shape_predictor = dlib.shape_predictor(self.predictor)
      self.facerec = dlib.face_recognition_model_v1(self.resnet_model)
      self.display = display

      self.reset()

      if self.display:
         self.win = dlib.image_window()

   def reset(self):

      self.filelist = []
      self.result_map = {}

   def fill_filelist(self):

      for subdir, dirs, files in os.walk(self.datapath):
         if(len(files) != 0):
            for f in files:
                filepath = subdir + os.sep + f

                if filepath.endswith(".jpg"):
                    self.filelist.append(filepath)

   def process_filelist(self):

      for f in self.filelist:
         img = io.imread(f)

         width = len(img)
         height = len(img[0])

         splitted_path = f.split("/")
         keyname = splitted_path[-3] + "_" + splitted_path[-2]
         #os.path.basename(os.path.dirname(f))
         

         if keyname not in self.result_map:
            self.result_map[keyname] = []

         if self.display:
            self.win.clear_overlay()
            self.win.set_image(img)

         dets = []
         if(splitted_path[-2] != "profile"):
            dets = self.detector(img, 1)
         else:
            dets.append(dlib.rectangle(0,height, width, 0))

         if len(dets) is 0:
            print("Warning, face detector return 0 for " + f)
            continue

         # Consider only one face (as is should be)
         d = dets[0]
         print("-----")
         print(d)

         print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(d, d.left(), d.top(), d.right(), d.bottom()))

         shape = self.shape_predictor(img, d)

         landmarks = np.matrix([[p.x, p.y] for p in shape.parts()])

         if self.display:
            self.win.clear_overlay()
            self.win.add_overlay(d)
            self.win.add_overlay(shape)



         print("Process: " + f)
         dlib_face_desc = self.facerec.compute_face_descriptor(img, shape)
         face_desc = [x for x in dlib_face_desc]

         self.result_map[keyname].append(face_desc)


   def save_result_map(self, filepath):

      with open(filepath,'w') as f:
         f.write(json.dumps(self.result_map))


def load_descriptor_map(filepath):

   with open(filepath) as f:
      result_str = f.read()

      result_all = json.loads(result_str)
      subject_all = result_all.keys()

      subject = [s for s in subject_all if s[-5:] != 'False']

      return subject, result_all

In [4]:
if __name__ == '__main__':
    
    parser = argparse.ArgumentParser(description= " Generate decriptors for each image")
    parser.add_argument("datapath", type = str, help="path to dataset")
    parser.add_argument("savepath", type= str, help = "path to save descriptor map")
    parser.add_argument('-f', dest = "force_overwrite", action = "store_true", help = "force to overwrite output")

    params = parser.parse_args()
    
    def precess_data(params):
        
        if os.path.exists(params.savepath) and not params.force_overwrite:
            print("warning: save path {} is aleady exist, run -f to force overwrite".format(params.savepath))
            exit(0)
        
        
        binarypath = "bin"
        predictor = "shape_predictor_68_face_landmarks.dat"
        resnet_model = "dlib_face_recognition_resnet_model_v1.dat"
        
        #output is  txt format
        
        
            
            
        
#    parser = argparse.ArgumentParser(description='Process image and generate descriptor')
#    parser.add_argument('datapath', type=str, help="path to data set")
#    parser.add_argument('savepath', type=str, help="path to save descriptor map")
#    parser.add_argument('-f', dest='force_overwrite', action='store_true', help="force overwrite output")
#    params = parser.parse_args()

#    def process_data(params):

#       if os.path.exists(params.savepath) and not params.force_overwrite:
#          print("Warning {} exists, run -f to force overwrite".format(params.savepath))
#          exit(0)

#       # datapath = "/media/alexandre/Seagate Backup Plus Drive/Alexandre/dataset/face_weighted_rules/dlib_only"
#       # datapath = "/home/alexandre/Development/dataset/faces/kogniz_production/persons-39-1527251861579"
#       binarypath = "bin"
#       mmod_model = "mmod_human_face_detector.dat"
#       predictor = "shape_predictor_5_face_landmarks.dat"
#       resnet_model = "dlib_face_recognition_resnet_model_v1.dat"
#       # outputpath = "tmp/encoded2.txt"

#       face_processor = DlibFaceProcessor(params.datapath, binarypath, predictor, resnet_model, True)

#       face_processor.fill_filelist()
#       face_processor.process_filelist()
#       face_processor.save_result_map(params.savepath)

#       print("written {}".format(params.savepath))

#    process_data(params)


usage: ipykernel_launcher.py [-h] [-f] datapath savepath
ipykernel_launcher.py: error: the following arguments are required: savepath


SystemExit: 2

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
